# Bubble Sort — Code-level Tracer (Jupyter)

This notebook shows a visual, code-level trace of Bubble Sort:
- Left: highlighted source code line that is executing
- Right: bar chart of the array state
- Controls: Generate, Play, Step, Reset, Size, Speed

Requirements:
-  matplotlib, numpy

Run the next cell to install missing packages (if needed), then run the visualization cell.

> **Note:** On some Linux systems, you may need to force the matplotlib backend to `'agg'` for this notebook to work. This is handled automatically in the visualization code.

> def _figure_to_html(self, arr_data: List[int], colors: List[str]) -> str:\n",
        "        # Use Agg backend directly for off-screen rendering\n",
        "        import matplotlib\n",
        "        matplotlib.use('agg', force=True)\n",

In [ ]:
# Install dependencies if you don't have them. Uncomment and run if needed.
# Note: you may need to restart the kernel after installing ipywidgets.
# !pip install ipywidgets matplotlib numpy

In [2]:
import textwrap
import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg
from IPython.display import display, clear_output
import ipywidgets as widgets
from dataclasses import dataclass
from typing import List
import io
import base64

# Source displayed on the left (keeps line numbers stable)
SOURCE = textwrap.dedent("""
def bubble_sort(a):
    n = len(a)
    if n <= 1:
        return a
    for passnum in range(n - 1):
        made_swap = False
        # inner loop: compare adjacent pairs
        for j in range(n - passnum - 1):
            # compare a[j] and a[j+1]
            if a[j] > a[j+1]:
                # swap them
                a[j], a[j+1] = a[j+1], a[j]
                made_swap = True
        # optional early exit
        if not made_swap:
            break
    return a
""").strip().splitlines()

@dataclass
class TraceFrame:
    """Immutable representation of a single trace frame."""
    action: str
    line: int
    passnum: int
    i: int
    j: int
    array: List[int]

def bubble_sort_trace(arr):
    """Yields trace frames while executing bubble sort on a copy of arr."""
    a = arr[:]
    n = len(a)
    yield TraceFrame('enter', 0, -1, -1, -1, a.copy())
    yield TraceFrame('enter', 1, -1, -1, -1, a.copy())
    if n <= 1:
        yield TraceFrame('done', 2, -1, -1, -1, a.copy())
        return
    for passnum in range(n - 1):
        yield TraceFrame('enter', 3, passnum, -1, -1, a.copy())
        made_swap = False
        yield TraceFrame('enter', 4, passnum, -1, -1, a.copy())
        for j in range(n - passnum - 1):
            yield TraceFrame('compare', 6, passnum, j, j+1, a.copy())
            if a[j] > a[j + 1]:
                a[j], a[j + 1] = a[j + 1], a[j]
                made_swap = True
                yield TraceFrame('swap', 8, passnum, j, j+1, a.copy())
        yield TraceFrame('pass_end', 10, passnum, -1, -1, a.copy())
        if not made_swap:
            break
    yield TraceFrame('done', 11, passnum if n > 1 else -1, -1, -1, a.copy())

def random_array(n):
    return [random.randint(1, max(1, n*4)) for _ in range(n)]

class ColorScheme:
    """Encapsulates color logic for visualization."""
    IDLE = '#4f83ff'
    COMPARE_ACTIVE = 'gold'
    SWAP_ACTIVE = 'crimson'
    SORTED = 'seagreen'
    EDGE = 'black'

    @staticmethod
    def get_colors(arr_len: int, action: str, i: int, j: int, passnum: int) -> List[str]:
        """Return bar colors based on action."""
        colors = [ColorScheme.IDLE] * arr_len
        if action == 'compare':
            if 0 <= i < arr_len:
                colors[i] = ColorScheme.COMPARE_ACTIVE
            if 0 <= j < arr_len:
                colors[j] = ColorScheme.COMPARE_ACTIVE
        elif action == 'swap':
            if 0 <= i < arr_len:
                colors[i] = ColorScheme.SWAP_ACTIVE
            if 0 <= j < arr_len:
                colors[j] = ColorScheme.SWAP_ACTIVE
        elif action == 'pass_end':
            tail_start = arr_len - (passnum + 1)
            for k in range(tail_start, arr_len):
                if 0 <= k < arr_len:
                    colors[k] = ColorScheme.SORTED
        elif action == 'done':
            colors = [ColorScheme.SORTED] * arr_len
        return colors

class HTMLRenderer:
    """Handles all HTML rendering."""
    CODE_STYLE = "background:#071127;padding:10px;border-radius:8px;color:#e6eef8;"
    CODE_TEXT_COLOR = "#dfefff"
    HIGHLIGHT_STYLE = "background:gold;padding:2px 6px;border-radius:4px;"

    @staticmethod
    def code_html(source_lines: List[str], highlight_line: int = None) -> str:
        """Generate highlighted source code HTML."""
        lines = []
        for idx, line in enumerate(source_lines):
            safe = line.replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;')
            num = f"{idx + 1:>2}"
            if idx == highlight_line:
                lines.append(f'<div style="{HTMLRenderer.HIGHLIGHT_STYLE}"><code style="font-family:monospace;">{num}: {safe}</code></div>')
            else:
                lines.append(f'<div><code style="font-family:monospace;color:{HTMLRenderer.CODE_TEXT_COLOR};">{num}: {safe}</code></div>')
        return f"<div style='{HTMLRenderer.CODE_STYLE}'>{''.join(lines)}</div>"

    @staticmethod
    def status_html(op: str, comps: int, swaps: int, passnum) -> str:
        """Generate status bar HTML."""
        return f"<div style='font-family:monospace;'><b>op:</b> {op} &nbsp;&nbsp; <b>comparisons:</b> {comps} &nbsp;&nbsp; <b>swaps:</b> {swaps} &nbsp;&nbsp; <b>pass:</b> {passnum}</div>"

class StatsCalculator:
    """Calculates algorithm statistics from trace steps."""
    @staticmethod
    def count_comparisons(steps: List[TraceFrame], up_to_idx: int) -> int:
        return sum(1 for k in range(up_to_idx + 1) if steps[k].action == 'compare')

    @staticmethod
    def count_swaps(steps: List[TraceFrame], up_to_idx: int) -> int:
        return sum(1 for k in range(up_to_idx + 1) if steps[k].action == 'swap')

class BubbleSortVisualizer:
    """Main visualization controller."""
    def __init__(self, n: int = 18, speed: int = 150):
        self.n = n
        self.speed = int(speed)
        self.arr = random_array(self.n)
        self.original = self.arr[:]
        self.steps = list(bubble_sort_trace(self.arr))
        self.frame = 0
        self._playing = False

        self._setup_widgets()
        self._attach_handlers()
        self.ui = self._build_ui()
        self._render_frame(0)

    def _setup_widgets(self):
        self.code_html = widgets.HTML(
            value=HTMLRenderer.code_html(SOURCE),
            layout=widgets.Layout(width='48%', height='360px', overflow='auto')
        )
        self.plot_html = widgets.HTML(
            value='<div style="width:100%; height:100%; display:flex; align-items:center; justify-content:center;">Loading...</div>',
            layout=widgets.Layout(width='52%', height='360px', border='1px solid #ccc')
        )
        self.status = widgets.HTML(value=HTMLRenderer.status_html("idle", 0, 0, '-'))

        self.btn_generate = widgets.Button(description='Generate', tooltip='Generate new array')
        self.btn_play = widgets.ToggleButton(description='Play', tooltip='Play / Pause')
        self.btn_step = widgets.Button(description='Step', tooltip='Advance one step')
        self.btn_reset = widgets.Button(description='Reset', tooltip='Reset to initial array')

    def _attach_handlers(self):
        self.btn_generate.on_click(lambda b: self._on_generate())
        self.btn_step.on_click(lambda b: self._on_step())
        self.btn_reset.on_click(lambda b: self._on_reset())
        self.btn_play.observe(self._on_play_toggle, names='value')

    def _build_ui(self):
        controls = widgets.HBox([self.btn_generate, self.btn_play, self.btn_step, self.btn_reset])
        top = widgets.HBox([self.code_html, self.plot_html])
        bottom = widgets.HBox([self.status])
        return widgets.VBox([controls, top, bottom])

    def _figure_to_html(self, arr_data: List[int], colors: List[str]) -> str:
        # Use Agg backend directly for off-screen rendering
        import matplotlib
        matplotlib.use('agg', force=True)
        plt.ioff()  # Ensure interactive mode is off
        fig, ax = plt.subplots(figsize=(5.5, 3.5))
        ax.bar(range(len(arr_data)), arr_data, color=colors, edgecolor=ColorScheme.EDGE)
        ax.set_ylim(0, max(arr_data) + max(1, int(max(arr_data) * 0.05)))
        ax.set_title("Array state", fontsize=12, fontweight='bold')
        ax.set_xlabel("Index")
        ax.set_ylabel("Value")
        fig.tight_layout()
        buf = io.BytesIO()
        fig.savefig(buf, format='png', dpi=80, bbox_inches='tight')
        buf.seek(0)
        img_base64 = base64.b64encode(buf.read()).decode()
        plt.close(fig)
        html = f'<img src="data:image/png;base64,{img_base64}" style="max-width:100%; height:auto;" />'
        return html

    def _on_generate(self):
        self.n = 18
        self.arr = random_array(self.n)
        self.original = self.arr[:]
        self.steps = list(bubble_sort_trace(self.arr))
        self._playing = False
        self._render_frame(0)

    def _on_step(self):
        self._playing = False
        idx = self.frame + 1
        if idx < len(self.steps):
            self._render_frame(idx)

    def _on_reset(self):
        self._playing = False
        self.arr = self.original[:]
        self.steps = list(bubble_sort_trace(self.arr))
        self._render_frame(0)

    def _on_play_toggle(self, change):
        if change['new']:
            self.btn_play.description = 'Pause'
            self._playing = True
            self._start_play()
        else:
            self.btn_play.description = 'Play'
            self._playing = False

    def _start_play(self):
        import threading, time
        def play_loop():
            while self._playing and self.frame < len(self.steps) - 1:
                self._render_frame(self.frame + 1)
                time.sleep(0.15)
            self._playing = False
            self.btn_play.value = False
        threading.Thread(target=play_loop, daemon=True).start()

    def _render_frame(self, idx: int):
        if not (0 <= idx < len(self.steps)):
            return
        self.frame = idx
        frame = self.steps[idx]
        colors = ColorScheme.get_colors(len(frame.array), frame.action, frame.i, frame.j, frame.passnum)
        self.plot_html.value = self._figure_to_html(frame.array, colors)
        self.code_html.value = HTMLRenderer.code_html(SOURCE, highlight_line=frame.line)
        comps = StatsCalculator.count_comparisons(self.steps, idx)
        swaps = StatsCalculator.count_swaps(self.steps, idx)
        self.status.value = HTMLRenderer.status_html(frame.action, comps, swaps, frame.passnum)

    def show(self):
        display(self.ui)

visualizer = BubbleSortVisualizer(n=18, speed=150)
visualizer.show()